# Query Understanding Agent LlamaPack

<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/llama-index-packs/llama-index-packs-query-understanding-agent/examples/query_understanding_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook shows you how to use our Query Understanding Agent LlamaPack.

This is a template showing you how to build an agent that can explicitly trigger a call for human intervention. In this specific agent, human intervention is triggered if the input query is not clear.

This is determined by having the agent pick a given tool using our `RouterQueryEngine`, getting back the response, and then using the response and tools to determine whether the question itself is clear.

If human intervention is needed, then a `HumanInputRequiredException` is thrown. We have an example showing how to use this exception to seek for additional human input/query clarification.

In [ ]:
from dotenv import load_dotenv

load_dotenv()

True

## Grab Data

In [ ]:
!mkdir -p 'data/paul_graham/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt' -O 'data/paul_graham/paul_graham_essay.txt'

--2024-03-07 21:54:37--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/paul_graham/paul_graham_essay.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8000::154, 2606:50c0:8001::154, 2606:50c0:8002::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8000::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 75042 (73K) [text/plain]
Saving to: ‘data/paul_graham/paul_graham_essay.txt’

data/paul_graham/pa 100%[===================>]  73.28K  --.-KB/s    in 0.01s   

2024-03-07 21:54:37 (5.91 MB/s) - ‘data/paul_graham/paul_graham_essay.txt’ saved [75042/75042]



In [ ]:
steve_jobs_text = """1995 was the year when Steve Jobs was approaching the end of his exile from Apple after being kicked out of the company a decade earlier. Him as his new company, NeXT, were brought back in to save Apple from near bankruptcy.Jul 25, 2018"""

## Load Data

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.response.pprint_utils import pprint_response
from llama_index.llms.openai import OpenAI
from llama_index.core.schema import Document

# Tool 1
llm = OpenAI()
data = SimpleDirectoryReader(input_dir="./data/paul_graham/").load_data()
index = VectorStoreIndex.from_documents(data)

# Tool 2
steve_index = VectorStoreIndex.from_documents([Document(text=steve_jobs_text)])

## Agents

In [ ]:
from llama_index.core.agent import AgentRunner
from llama_index.packs.query_understanding_agent.step import (
    QueryUnderstandingAgentWorker,
    HumanInputRequiredException,
)
from llama_index.core.tools import QueryEngineTool

llm = OpenAI(model="gpt-4")

tools = [
    QueryEngineTool.from_defaults(
        query_engine=index.as_query_engine(),
        description="A tool that is useful for retrieving specific snippets from the Paul Graham's life",
    ),
    QueryEngineTool.from_defaults(
        query_engine=steve_index.as_query_engine(),
        description="A tool that is useful for retrieving specific snippets from the Steve Jobs's life",
    ),
]

## Baseline

This does not programmatically pause to detect ambiguity to allow user to provide input. Not only that, sometimes this hallucinate to a random subject.

In [ ]:
from llama_index.core.agent import ReActAgentWorker

callback_manager = llm.callback_manager
agent_worker = ReActAgentWorker.from_tools(
    tools,
    llm=llm,
    verbose=True,
    callback_manager=callback_manager,
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)
orig_question = "What did the author do in the summer of 1995?"
response = agent.chat(orig_question)

Thought: The user is asking about the author but hasn't specified which author. I have tools to query information about Paul Graham and Steve Jobs. I need to ask the user to specify the author.

Action: None
Answer: Could you please specify which author you are referring to? I have information about Paul Graham and Steve Jobs.


## Query Understanding Agent Worker

This allows the agent to ask user for clarification if the user query is unclear

In [ ]:
# from llama_index.core.agent import ReActAgentWorker
callback_manager = llm.callback_manager
agent_worker = QueryUnderstandingAgentWorker.from_tools(
    tools,
    llm=llm,
    callback_manager=callback_manager,
)
agent = AgentRunner(agent_worker, callback_manager=callback_manager)

In [ ]:
# from llama_index.llms.openai import OpenAI

orig_question = "what did he do in the summer of 1995?"
llm = OpenAI(model="gpt-4")
clarifying_questions = []

try:
    response = agent.chat(orig_question)
except HumanInputRequiredException as e:
    response = input(e.message)
    clarifying_questions.append((e.message, response))
    should_end = False
    while not should_end:
        clarifying_texts = "\n".join(
            [
                f"""
   Q: {question}
   A: {answer}
        """
                for question, answer in clarifying_questions
            ]
        )
        query_text = f"""
Given a query and a set of clarifying questions, please rewrite the query to be more clear.
Example:
Q: What trajectory is the monthly earning from the three months: April, May and June?
Clarifying Questions:
   Q: What year are you referring to?
   A: In 2022
   Q: What company are you referring to?
   A: Uber
Rewrite: What was the trajectory of Uber's monthly earnings for the months of April, May, and June in 2022?

Q:{orig_question}
Clarifying Questions: {clarifying_texts}
Rewrite: """
        rewrite_response = llm.complete(query_text)
        orig_question = rewrite_response
        try:
            output = agent.chat(rewrite_response.text)
            should_end = True
            print(f"response: {output.response}")
        except HumanInputRequiredException as er:
            response = input(er.message)
            clarifying_questions.append((er.message, response))

response: In the summer of 1995, Steve Jobs was involved in the process of returning to Apple after his departure from the company a decade earlier.
